In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import gc
import os
from scipy.stats import spearmanr
from math import floor, ceil

np.set_printoptions(suppress=True)

In [2]:
PATH = '../input/google-quest-challenge/'
BERT_PATH = '../input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12'
tokenizer = tokenization.FullTokenizer(BERT_PATH+'/assets/vocab.txt', True)
MAX_SEQUENCE_LENGTH = 512

df_train = pd.read_csv(PATH+'train.csv')
df_test = pd.read_csv(PATH+'test.csv')
df_sub = pd.read_csv(PATH+'sample_submission.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

output_categories = list(df_train.columns[11:])
input_categories = list(df_train.columns[[1,2,5]])
print('\noutput categories:\n\t', output_categories)
print('\ninput categories:\n\t', input_categories)

train shape = (6079, 41)
test shape = (476, 11)

output categories:
	 ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 'answer_well_written']

input categories:
	 ['question_title', 'question_body', 'answer']


In [3]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids,input_masks,input_segments]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer

        t, q, a = _trim_input(t, q, a, max_sequence_length)

        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return np.asarray(input_ids, dtype=np.int32)


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [4]:
inputs = compute_input_arrays(df_train , input_categories , tokenizer , MAX_SEQUENCE_LENGTH)
outputs = compute_output_arrays(df_train, output_categories)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [5]:
inputs.shape , df_train.shape 

((6079, 512), (6079, 41))

In [6]:
inputs_n = np.asarray([i / 40000 for i in inputs])
test_inputs_n = np.asarray([i / 40000 for i in test_inputs])

In [7]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(1000),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(500 , activation='sigmoid'),
                             tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(30 , activation='relu')
])

In [8]:
model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['accuracy'])
model.fit(inputs , outputs , epochs=10)

Train on 6079 samples
Epoch 1/10
6079/6079 [==============================] - 3s 472us/sample - loss: 2.6871 - accuracy: 0.5247
Epoch 2/10
6079/6079 [==============================] - 2s 266us/sample - loss: 2.8379 - accuracy: 0.5292
Epoch 3/10
6079/6079 [==============================] - 2s 266us/sample - loss: 3.1662 - accuracy: 0.5363
Epoch 4/10
6079/6079 [==============================] - 2s 260us/sample - loss: 3.0436 - accuracy: 0.5297
Epoch 5/10
6079/6079 [==============================] - 2s 272us/sample - loss: 2.9930 - accuracy: 0.5272
Epoch 6/10
6079/6079 [==============================] - 2s 285us/sample - loss: 3.3330 - accuracy: 0.5163
Epoch 7/10
6079/6079 [==============================] - 2s 287us/sample - loss: 3.3527 - accuracy: 0.5293
Epoch 8/10
6079/6079 [==============================] - 2s 272us/sample - loss: 3.2185 - accuracy: 0.5335
Epoch 9/10
6079/6079 [==============================] - 2s 267us/sample - loss: 3.3785 - accuracy: 0.5280
Epoch 10/10
6079/6079 [=

In [9]:
a = model.predict(test_inputs)

In [10]:
a1 = [i/10 for i in a]

In [11]:
a2 = np.asarray(a1)

In [12]:
df_sub.iloc[:, 1:] = a2
df_sub.to_csv('submission.csv', index=False)